In [37]:
import pandas as pd
import pickle
import sys
import os
sys.path.append(os.path.abspath(os.path.join('../modules')))

In [2]:
from clean_transcript import CleanTrans
from pre_process import PrepSound

In [3]:
train_path = '../data/SWAHILI/data/train/wav'
test_path = '../data/SWAHILI/data/test/wav5'

In [4]:
train_folders = os.listdir(train_path)
test_folder = os.listdir(test_path)

In [5]:
'''specs_dict = {'waves':[], 'specs':[], 'sample_len':[], 'processed_len':[], 'smaple_rate':[]}
num_i = 0
while num_i < 2000:
    for folder in train_folders:
        folder_path = train_path+'/'+folder
        for f in os.listdir(folder_path):
            if num_i < 2000:
                if f.endswith('.wav'):
                  prep = PrepSound(audio_file=f, path=folder_path)
                  prep_out = prep.pre_process(newsr=16000, max_ms=58400)
                  specs_dict['waves'].append(f[:-4])
                  specs_dict['specs'].append(prep_out['specs'])
                  specs_dict['sample_len'].append(prep_out['sample_len'])
                  specs_dict['processed_len'].append(prep_out['processed_len'])
                  specs_dict['smaple_rate'].append(prep_out['smaple_rate'])
                  num_i += 1
                else:
                    break'''

"'specs_dict = {'waves':[], 'specs':[], 'sample_len':[], 'processed_len':[], 'smaple_rate':[]}\nnum_i = 0\nwhile num_i < 2000:\n    for folder in train_folders:\n        folder_path = train_path+'/'+folder\n        for f in os.listdir(folder_path):\n            if num_i < 2000:\n                if f.endswith('.wav'):\n                  prep = PrepSound(audio_file=f, path=folder_path)\n                  prep_out = prep.pre_process(newsr=16000, max_ms=58400)\n                  specs_dict['waves'].append(f[:-4])\n                  specs_dict['specs'].append(prep_out['specs'])\n                  specs_dict['sample_len'].append(prep_out['sample_len'])\n                  specs_dict['processed_len'].append(prep_out['processed_len'])\n                  specs_dict['smaple_rate'].append(prep_out['smaple_rate'])\n                  num_i += 1\n                else:\n                    break"

In [40]:
'''specs_dict_all = {'waves':[], 'specs':[], 'sample_len':[], 'processed_len':[], 'smaple_rate':[]}
for folder in train_folders:
    folder_path = train_path+'/'+folder
    for f in os.listdir(folder_path):
        if f.endswith('.wav'):
          prep = PrepSound(audio_file=f, path=folder_path)
          prep_out = prep.pre_process(newsr=16000, max_ms=58400)
          specs_dict_all['waves'].append(f[:-4])
          specs_dict_all['specs'].append(prep_out['specs'])
          specs_dict_all['sample_len'].append(prep_out['sample_len'])
          specs_dict_all['processed_len'].append(prep_out['processed_len'])
          specs_dict_all['smaple_rate'].append(prep_out['smaple_rate'])'''

In [45]:
test_folder

['.DS_Store',
 '._.DS_Store',
 'SWH-05-20101124',
 'SWH-05-20101211',
 'SWH-05-20101222',
 'SWH-15-20110203',
 'SWH-15-20110311',
 'SWH-15-20110323']

In [49]:
"""test_specs_dict_all = {'waves':[], 'specs':[], 'sample_len':[], 'processed_len':[], 'smaple_rate':[]}
for folder in test_folder[2:]:
    folder_path = test_path+'/'+folder
    for f in os.listdir(folder_path):
        if f.endswith('.wav'):
          prep = PrepSound(audio_file=f, path=folder_path)
          prep_out = prep.pre_process(newsr=16000, max_ms=58400)
          test_specs_dict_all['waves'].append(f[:-4])
          test_specs_dict_all['specs'].append(prep_out['specs'])
          test_specs_dict_all['sample_len'].append(prep_out['sample_len'])
          test_specs_dict_all['processed_len'].append(prep_out['processed_len'])
          test_specs_dict_all['smaple_rate'].append(prep_out['smaple_rate'])"""

RuntimeError: Error loading audio file: failed to open file ../data/SWAHILI/data/test/wav5/SWH-05-20101124/._16k-emission_swahili_05h30_-_06h00_tu_20101124_part001g.wav

In [ ]:
pickle.dump(test_specs_dict_all, open('../data/test_dict_all', 'wb'))

In [42]:
#pickle.dump(specs_dict_all, open('../data/train_dict_all', 'wb'))

In [6]:
#pickle.dump(specs_dict, open('../data/train_sample_dict', 'wb'))

In [7]:
train_text_path = '../data/SWAHILI/data/train/text'
test_text_path = '../data/SWAHILI/data/test/text'
transcript = pd.read_csv(train_text_path, sep='\t', names=['file','transcript'])

In [8]:
transcript['file'][0]

'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part10'

In [9]:
transcript['file'][1]

'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part100'

In [10]:
from clean_transcript import CleanTrans

In [11]:
clean = CleanTrans(df=transcript, trans_col='transcript')

In [12]:
transcript_clean = clean.run_all()

In [13]:
transcript_clean

,file,transcript
0,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,rais wa tanzania jakaya mrisho kikwete
1,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,yanayo andaliwa nami pendo pondo idhaa ya kisw...
2,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,inayokutangazia moja kwa moja kutoka jijini da...
3,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,juma hili bara la afrika limeshuhudia raia wa ...
4,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,wakipiga kura ya maoni ilikufanya mabadiliko ya
...,...,...
10105,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,na somo lile lililopokelewa kule kenya
10106,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,ambapo mtu aliyeshindwa kwenye uchaguzi
10107,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,ni kauli yake mchambuzi wa masuala ya siasa
10108,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,mwanasheria anayemtetea rais wa zamani wa liberia


In [14]:
specs_dict_l = pickle.load(open('../data/train_sample_dict', 'rb'))

In [15]:
from tokenize_text import TokenizerText

In [16]:
texts_list = [t.split(' ') for t in transcript_clean['transcript']]

In [17]:
tokenize = TokenizerText(texts=transcript_clean['transcript'],padding='pre',filters=None, reverse=False)

In [18]:
tokens=[tokenize.text_to_tokens(text=t, reverse=False, padding=True) for t in transcript_clean['transcript']]

In [19]:
transcript_clean['tokens'] = tokens

In [20]:
spec_dict_df = pd.DataFrame(specs_dict_l)

In [21]:
spec_dict_df.head()

,waves,specs,sample_len,processed_len,smaple_rate
0,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,"[[[tensor(-24.5810), tensor(-24.5810), tensor(...",50240,934400,16000
1,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,"[[[tensor(-26.9436), tensor(-26.9436), tensor(...",49600,934400,16000
2,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,"[[[tensor(-24.4068), tensor(-24.4068), tensor(...",58400,934400,16000
3,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,"[[[tensor(-23.7175), tensor(-23.7175), tensor(...",62400,934400,16000
4,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,"[[[tensor(-26.2334), tensor(-26.2334), tensor(...",47040,934400,16000


In [22]:
#spec_dict_df['waves'] == 
transcript_clean['file'][4]

'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part103'

In [23]:
spec_dict_df['waves'][4]

'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part103'

In [24]:
train_df = spec_dict_df.merge(transcript_clean, left_on='waves', right_on='file', how='inner')
train_df.drop(columns='file', inplace=True)

In [25]:
train_df['waves'][5] == train_df['waves'][0]

False

In [26]:
train_df[['transcript', 'tokens']]

,transcript,tokens
0,rais wa tanzania jakaya mrisho kikwete,"[[17, 1, 3, 13, 2, 9, 1, 2, 12, 1, 4, 18, 1, 4..."
1,yanayo andaliwa nami pendo pondo idhaa ya kisw...,"[[15, 1, 4, 1, 15, 10, 2, 1, 4, 19, 1, 14, 3, ..."
2,inayokutangazia moja kwa moja kutoka jijini da...,"[[3, 4, 1, 15, 10, 5, 6, 12, 1, 4, 20, 1, 18, ..."
3,juma hili bara la afrika limeshuhudia raia wa ...,"[[21, 6, 7, 1, 2, 11, 3, 14, 3, 2, 16, 1, 17, ..."
4,wakipiga kura ya maoni ilikufanya mabadiliko ya,"[[9, 1, 5, 3, 23, 3, 20, 1, 2, 5, 6, 17, 1, 2,..."
...,...,...
1972,timu kama red berates pia ilikuwa hivyo hivyo,"[[12, 3, 7, 6, 2, 5, 1, 7, 1, 2, 17, 8, 19, 2,..."
1973,ilikuwa na ile hali ya kuchanganya changanya w...,"[[3, 14, 3, 5, 6, 9, 1, 2, 4, 1, 2, 3, 14, 8, ..."
1974,hiyo ndio pia ilikuja ikachangia wakakuja waka...,"[[11, 3, 15, 10, 2, 4, 19, 3, 10, 2, 23, 3, 1,..."
1975,unajua sasa ikifika pale katikati sasa hapo nd...,"[[6, 4, 1, 21, 6, 1, 2, 13, 1, 13, 1, 2, 3, 5,..."


##**Load audio file**


In [27]:
len(transcript_clean)

10110

In [35]:
arr = tokenize.text_to_tokens(text='andaliwa andaliwa')
tex = tokenize.tokens_to_string(tokens=arr)
alph = tokenize.token_to_word(token=8)

In [36]:
alph

'e'

##**Load transcriptions**

##**Convert into channels** 
Some of the sound files are mono (ie. 1 audio channel) while most of them are stereo (ie. 2 audio channels). Since the Neural network model expects all items to have the same dimensions, we will convert the mono files to stereo, by duplicating the first channel to the second


##**Standardize sampling rate**
We must standardize and convert all audio to the same sampling rate so that all arrays have the same dimensions.

##**Resize to the same length**
Resize to get an equal-sized audio sample by extending duration by padding it with silence, or by truncating it.

##**Data argumentation**
Perform data augmentation on the raw audio signal by applying a Time Shift to shift the audio to the left or the right by a random amount. 

##**Feature extraction**: 
Speech recognition methods derive features from the audio, such as Spectrogram or Mel Frequency Cepstrum (MFCC).

* Convert the augmented audio to a Mel Spectrogram.








**Acoustic modeling:** 

After features are extracted, these vectors are passed to acoustic models. An acoustic model attempts to map the audio signal to the basic units of speech such as phonemes or graphemes.